In [ ]:
import pyam
import nomenclature

In [ ]:
df_1 = pyam.IamDataFrame("source/8_Paris_compatible/Chapter8_Fig1_final_data 1.xlsx")
df_2 = pyam.IamDataFrame("source/8_Paris_compatible/Chapter8_Fig2_final_data 2.xlsx")
df_4 = pyam.IamDataFrame("source/8_Paris_compatible/Chapter8_Fig4_final_data 2.xlsx")

In [ ]:
df_4.variable

In [ ]:
variable_mapping = dict([
        (
            i, 
            i.replace("Conventional CDR", "Carbon Flux [Land]|CDR")
            .replace("Inventory-Aligned Carbon Removal", "Carbon Flux [Land]")
            .replace("Additional Inventory Carbon Removal", "Carbon Flux [Land]|Indirect")
        )
    for i in df_4.variable])

In [ ]:
df_4.rename(
    variable=dict([
        (
            i, 
            i.replace("Conventional CDR", "Carbon Flux [Land]|CDR")
            .replace("Inventory-Aligned Carbon Removal", "Carbon Flux [Land]")
            .replace("Additional Inventory Carbon Removal", "Carbon Flux [Land]|Indirect")
        )
    for i in df_4.variable]),
    inplace=True,
)

In [ ]:
df_4.rename(
    scenario={
        "2020 Modelled values":  "All Assessed Scenarios",
        "2050 C1": "C1",
        "2050 C3": "C3",
        "2050 Current Policies": "Current Policies",
    },
    inplace=True,
)

In [ ]:
df = pyam.concat(
    [df_1, df_2, df_4]
)

In [ ]:
df.rename(
    variable=dict(
        [
            (
                i, 
                i.replace("Conventional CDR", "Carbon Removal|Conventional")
                .replace("Novel CDR", "Carbon Removal|Novel")
                .replace("CDR", "Carbon Removal")
                .replace("Emissions|Kyoto Gases|Gross", "Gross Emissions|Kyoto Gases").
                replace("[p50]", "[Median]")
            )
        for i in df.variable]
    ),
    inplace=True,
)

df.rename(unit={"Gt CO2-equiv/yr": "Gt CO2e/yr"}, inplace=True)

In [ ]:
dsd = nomenclature.DataStructureDefinition("../definitions/")

In [ ]:
df.rename(
    model={"SoCDR_v2": "State of CDR (2024) - Paris-Consistent Pathways"},
    inplace=True
)

In [ ]:
carbon_removal_df = df.filter(variable="Carbon Removal*")

In [ ]:
carbon_removal_df.rename(unit={"Gt CO2e/yr": "Gt CO2/yr"}, inplace=True)

In [ ]:
df = df.filter(variable="Carbon Removal*", keep=False).append(carbon_removal_df)

In [ ]:
df.variable

In [ ]:
df.scenario

In [ ]:
df.rename(
    scenario={
        "C1": "1.5C Low and No Overshoot (>50%) [C1]",
        "C2": "1.5C with High Overshoot (>50% in 2100) [C2]",
        "C3": "2C (>67%) [C3]",
        "C1 - C3": "1.5C (>50%) to 2C (>67%) [C1,C2,C3]",
    },
    inplace=True,
)

In [ ]:
df

In [ ]:
dsd.validate(df)

In [ ]:
df.set_meta(meta="Chapter 8", name="Chapter")

In [ ]:
df.filter(scenario="*[C1,C2,C3]").scenario

In [ ]:
df.scenario

In [ ]:
df.filter(scenario="*[C1,C2,C3]").to_ixmp4("socdr-dev")

In [ ]:
platform.units.create("Gt CO2e/yr")

In [ ]:
import ixmp4

In [ ]:
platform = ixmp4.Platform("socdr-dev")

In [ ]:
for model, scenario in df.index:
    try:
        run = platform.runs.get(model, scenario)
        run.iamc.remove(run.iamc.tabulate())
        run.iamc.add(df.filter(model=model, scenario=scenario).data)
    except:
        df.filter(model=model, scenario=scenario).to_ixmp4("socdr-dev")
        print("Created " + model + " - " + scenario)        


In [ ]:
model

In [ ]:
df_online = platform.iamc.tabulate(model=model, run=dict(default_only=True))

In [ ]:
duplicates = df_online[df_online.duplicated()]

In [ ]:
list(duplicates.scenario.unique())

In [ ]:
df.scenario

In [ ]:
df_online = pyam.read_ixmp4("socdr-dev", model=model)

In [ ]:
df

In [ ]:
df_online

In [ ]:
df.to_excel("chapter-8-data.xlsx")